<a href="https://colab.research.google.com/github/RVijayChiranjithReddy/Autolabel_encoder/blob/main/conditional_GAN_auto_label_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.initializers import RandomNormal
from keras.applications import *
import tensorflow as tf
import matplotlib.pyplot as plt
import keras.backend as K
from keras.optimizers import RMSprop, SGD, Adam
import cv2
from PIL import Image
import numpy as np
import glob
from random import randint, shuffle
%matplotlib inline
K.set_learning_phase(1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
df = pandas.read_csv('/content/styles.csv')
df.fillna('DK', inplace = True) 
data = {}
for i in range(44446):
  tags = [str(x) for x in df.loc[i].values[1:9] if type(x) is str]
  #desc = df.loc[i].values[9]
  #if type(desc) is not float:
    #for de in desc.split():
     #if de.isalpha() and de not in tags:
        #tags.append(de)
  data[str(df.loc[i].values[0])] = ':'.join(tags)
vocab = []
for k,v in data.items():
  temp = list(set(v.split(':')))
  vocab = vocab + temp
  vocab = list(set(vocab))
print(len(vocab))
#from tensorflow.keras.preprocessing.text import one_hot
#for k,d in data.items():
  #data_encoded[k] = one_hot(' '.join(d),len(vocab))
tokenizer1 = Tokenizer(num_words=25)
tokenizer2 = Tokenizer(num_words=25)
tokenizer3 = Tokenizer(num_words=54)
tokenizer4 = Tokenizer(num_words=173)
tokenizer5 = Tokenizer(num_words=48)
tokenizer6 = Tokenizer(num_words=25)
tokenizer7 = Tokenizer(num_words=25)
# This builds the word index
tokenizer1.fit_on_texts(df['gender'])
tokenizer2.fit_on_texts(df['masterCategory'])
tokenizer3.fit_on_texts(df['subCategory'])
tokenizer4.fit_on_texts(df['articleType'])
tokenizer5.fit_on_texts(df['baseColour'])
tokenizer6.fit_on_texts(df['season'])
tokenizer7.fit_on_texts(df['usage'])


# This turns strings into lists of integer indices.
#train_sequences = tokenizer.texts_to_sequences(data.values())
print(tokenizer1.word_index)
print(tokenizer2.word_index)
print(tokenizer3.word_index)
print(tokenizer4.word_index)
print(tokenizer5.word_index)
print(tokenizer6.word_index)
print(tokenizer7.word_index)

240
{'men': 1, 'women': 2, 'unisex': 3, 'boys': 4, 'girls': 5}
{'apparel': 1, 'accessories': 2, 'footwear': 3, 'personal': 4, 'care': 5, 'free': 6, 'items': 7, 'sporting': 8, 'goods': 9, 'home': 10}
{'topwear': 1, 'shoes': 2, 'bags': 3, 'bottomwear': 4, 'watches': 5, 'innerwear': 6, 'jewellery': 7, 'eyewear': 8, 'fragrance': 9, 'sandal': 10, 'wallets': 11, 'flip': 12, 'flops': 13, 'belts': 14, 'socks': 15, 'lips': 16, 'and': 17, 'dress': 18, 'loungewear': 19, 'nightwear': 20, 'saree': 21, 'nails': 22, 'makeup': 23, 'headwear': 24, 'ties': 25, 'accessories': 26, 'skin': 27, 'scarves': 28, 'cufflinks': 29, 'apparel': 30, 'set': 31, 'free': 32, 'gifts': 33, 'stoles': 34, 'care': 35, 'eyes': 36, 'mufflers': 37, 'shoe': 38, 'sports': 39, 'equipment': 40, 'gloves': 41, 'hair': 42, 'bath': 43, 'body': 44, 'water': 45, 'bottle': 46, 'perfumes': 47, 'umbrellas': 48, 'beauty': 49, 'wristbands': 50, 'home': 51, 'furnishing': 52, 'vouchers': 53}
{'tshirts': 1, 'shoes': 2, 'shirts': 3, 'casual': 4,

In [ ]:
print(data['48311'].split(':')[4])
print(tokenizer5.texts_to_sequences([['bronze']]))

Bronze
[[32]]


In [ ]:
da = {}
ds = {}
for k,v in data.items():
  da[str(k)] =[]
  da[str(k)].append(tokenizer1.texts_to_sequences([v.split(':')[0]])[0][0])
  da[str(k)].append(tokenizer2.texts_to_sequences([v.split(':')[1]])[0][0])
  da[str(k)].append(tokenizer3.texts_to_sequences([v.split(':')[2]])[0][0])
  da[str(k)].append(tokenizer4.texts_to_sequences([v.split(':')[3]])[0][0])
  da[str(k)].append(tokenizer5.texts_to_sequences([v.split(':')[4]])[0][0])
  da[str(k)].append(tokenizer6.texts_to_sequences([v.split(':')[5]])[0][0])
  da[str(k)].append(tokenizer7.texts_to_sequences([v.split(':')[6]])[0][0])
  i=i+1


In [ ]:
def generator(img_shape):
  inputs = Input(shape= img_shape)
  x  = Conv2D(64,padding = 'same',strides = 4,kernel_size = 3)(inputs)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  x1 = MaxPooling2D((3,3),strides =(1,1),padding='same')(x)
  x1  = Conv2D(64,padding = 'same',strides = 1,kernel_size = 1)(x1)
  x1 = BatchNormalization()(x1)
  x1 = LeakyReLU(alpha=0.2)(x1)
  x3  = Conv2D(64,padding = 'same',strides = 1,kernel_size = 1)(x)
  x3  = Conv2D(64,padding = 'same',strides = 1,kernel_size = 3)(x3)
  x3 = BatchNormalization()(x3)
  x3 = LeakyReLU(alpha=0.2)(x3)
  x5  = Conv2D(64,padding = 'same',strides = 1,kernel_size = 1)(x)
  x5  = Conv2D(64,padding = 'same',strides = 1,kernel_size = 5)(x5)
  x5 = BatchNormalization()(x5)
  x5 = LeakyReLU(alpha=0.2)(x5)
  x_135 = concatenate([x,x1,x3,x5],axis = -1)
  x21  = Conv2D(32,padding = 'same',strides = 2,kernel_size = 3)(x_135)
  x21 = BatchNormalization()(x21)
  x21 = LeakyReLU(alpha=0.2)(x21)
  x22 = MaxPooling2D((3,3),strides =(1,1),padding='same')(x21)
  x22  = Conv2D(32,padding = 'same',strides = 1,kernel_size = 1)(x22)
  x22 = BatchNormalization()(x22)
  x22 = LeakyReLU(alpha=0.2)(x22)
  x23  = Conv2D(32,padding = 'same',strides =1,kernel_size = 1)(x21)
  x23  = Conv2D(32,padding = 'same',strides = 1,kernel_size = 3)(x23)
  x23 = BatchNormalization()(x23)
  x23 = LeakyReLU(alpha=0.2)(x23)
  x25  = Conv2D(32,padding = 'same',strides = 1,kernel_size = 1)(x21)
  x25  = Conv2D(32,padding = 'same',strides = 1,kernel_size = 3)(x25)
  x25 = BatchNormalization()(x25)
  x25 = LeakyReLU(alpha=0.2)(x25)
  x_2135 = concatenate([x21,x22,x23,x25],axis=-1)
  x  = Conv2D(64,padding = 'same',strides = 2,kernel_size = 3)(x_2135)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  x  = Conv2D(32,padding = 'same',strides = 2,kernel_size = 3)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  x  = Conv2D(16,padding = 'same',strides = 2,kernel_size = 3)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = Flatten()(x)
  x = Dense(4096,activation = 'relu')(x)
  x = Dense(1024,activation = 'relu')(x)
  x = Dense(7,activation = 'relu')(x)
  return Model(inputs= inputs,outputs = x)
g = generator((256,256,3))
g.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 64, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
batch_size = 64
def cgan_gen():
  input1 = Input(shape= (256,256,3))
  input2 = Input(shape = (7,))
  i = Embedding(input_dim=239,output_dim=8,input_length=13)(input2)
  x = Flatten()(i)
  gener = generator(img_shape=(256,256,3))
  x1 = gener(input1)
  lt = Dense(56,activation='relu')(x1)
  x = Add()([lt,x])
  x = Dense(4096,activation = 'relu')(x)
  x = BatchNormalization()(x)
  x = Dense(1024,activation = 'relu')(x)
  x = BatchNormalization()(x)
  x = Dense(239,activation = 'relu')(x)
  x = Dense(7,activation = 'relu')(x)
  return Model(inputs = [input1,input2],outputs = x)
c = cgan_gen()
c.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, 7)            5603127     input_2[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 7, 8)         1912        input_3[0][0]                    
____________________________________________________________________________________________

In [ ]:
def discriminator():
  x1 = Input(shape = (256,256,3))
  x2 = Input(shape=(7,))
  discr = generator(img_shape=(256,256,3))
  xn = discr(x1)
  xn = Dense(56,activation = 'relu')(xn)
  i = Embedding(input_dim=len(vocab),output_dim=8,input_length=7)(x2)
  x = Flatten()(i)
  xc = concatenate([xn,x])
  x = Dense(4096,activation='relu')(xc)
  x = BatchNormalization(momentum=0.9)(x)
  x = Dense(4096,activation='relu')(x)
  x= BatchNormalization(momentum=0.9)(x)
  x = Dense(1,activation='sigmoid')(x)
  return Model(inputs = [x1,x2],outputs = x)

In [ ]:
def gan(disc,cgan):
  x1 = Input(shape = (256,256,3))
  x2 = Input(shape = (7,))
  xc = cgan([x1,x2]) 
  x = disc([x1,xc])
  return Model([x1,x2],x)
disc = discriminator()
disc.compile(optimizer = Adam(lr=0.001),loss ='binary_crossentropy',metrics=['accuracy'])
cgan = cgan_gen()
#cgan.compile(optimizer = Adam(lr=0.001),loss ='categorical_crossentropy',metrics=['accuracy'])
disc.trainable = False
GAN = gan(disc,cgan)
GAN.compile(loss = 'binary_crossentropy',optimizer = Adam(lr=0.001),metrics=['accuracy'])
GAN.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 7)]          0                                            
__________________________________________________________________________________________________
model_6 (Functional)            (None, 7)            10301422    input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
model_4 (Functional)            (None, 1)            22886520    input_11[0][0]             

In [ ]:
def read_image(fn):
  im = cv2.imread(fn)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2LAB)
  im = cv2.resize(im, (256,256), interpolation=cv2.INTER_CUBIC)
  im = np.array(im)/255*2-1
  return im

In [ ]:
traini = glob.glob(r'/content/drive/MyDrive/images/*')
def minibatch(data, batchsize):
    length = len(data)
    epoch = i = 0
    tmpsize = None    
    while True:
        size = tmpsize if tmpsize else batchsize
        if i+size > length:
            shuffle(data)
            i = 0
            epoch+=1        
        rtn = [read_image(data[j]) for j in range(i,i+size)]
        ltn=[]
        se = []
        for j in range(i,i+size):
          if '(' in data[j].split('/')[-1].split('.')[0]:
            t=[]
            p = data[j].split('/')[-1].split('.')[0].split(' ')[0]
            t.append(da[p][0]/5)
            t.append(da[p][1]/10)
            t.append(da[p][2]/53)
            t.append(da[p][3]/172)
            t.append(da[p][4]/47)
            t.append(da[p][5]/5)
            t.append(da[p][6]/9)
            se.append(t) 
          else:
            t=[]
            p = data[j].split('/')[-1].split('.')[0]
            t.append(da[p][0]/5)
            t.append(da[p][1]/10)
            t.append(da[p][2]/53)
            t.append(da[p][3]/172)
            t.append(da[p][4]/47)
            t.append(da[p][5]/5)
            t.append(da[p][6]/9)
            se.append(t) 
        #ltn = [data_enc_padded[data[j].split('/')[-1].split('.')[0]] for j in range(i,i+size)]
        i+=size
        tmpsize = yield epoch, np.float32(rtn), np.array(se)

def minibatchAB(dataA, batchsize):
    batchA = minibatch(dataA, batchsize)
    tmpsize = None    
    while True:        
        ep1, A ,B = batchA.send(tmpsize)
        tmpsize = yield ep1, A , B

In [ ]:
def train(iterations, batch_size, sample_interval):
  train_batch = minibatchAB(traini,128)
  for iteration in range(iterations):
    for i in range(1):
      epoch, A,B= next(train_batch)
      z = np.random.rand(batch_size, 7)
      f = cgan.predict([A,z])
      d_loss_fake = disc.train_on_batch([A,f],np.zeros(shape=(batch_size,1)))
      d_loss_real = disc.train_on_batch([A,B],np.ones(shape=(batch_size,1)))
      print('hi')   
    d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
    epoch, A,B = next(train_batch)
    optimizer = tf.keras.optimizers.Adam(lr=0.01)
    #g_loss = cgan.train_on_batch([A,z],S)
    #g_loss = GAN.train_on_batch([A,z],np.ones(shape=(batch_size,1))) 
    with tf.GradientTape() as tape:
      z = np.random.rand(batch_size, 7)
      y = GAN([A,z])
      loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(np.ones(shape=(batch_size,1)),y))
      varis = cgan.trainable_variables + disc.variables
      grads = tape.gradient(loss, varis)
      optimizer.apply_gradients(zip(grads,varis))
    if (iteration + 1) % sample_interval == 0:
      print("%d [D loss: %f, acc.: %.2f%%]" % (iteration + 1, np.sum(d_loss), 100.0 * np.sum(accuracy)))
      print(tf.reduce_sum(loss))
      cgan.save('/content/drive/MyDrive/cgan.h5')
      disc.save('/content/drive/MyDrive/disc.h5')

In [ ]:
train(2000,128,1)

hi
1 [D loss: 0.810956, acc.: 49.61%]
tf.Tensor(8.6700677e-13, shape=(), dtype=float32)
hi
2 [D loss: 0.723553, acc.: 50.00%]
tf.Tensor(8.624272e-07, shape=(), dtype=float32)
hi
3 [D loss: 0.763641, acc.: 48.83%]
tf.Tensor(5.5028574e-24, shape=(), dtype=float32)
hi
4 [D loss: 0.756183, acc.: 50.00%]
tf.Tensor(6.6758535e-20, shape=(), dtype=float32)
hi
5 [D loss: 0.710956, acc.: 50.00%]
tf.Tensor(5.454365e-13, shape=(), dtype=float32)
hi
6 [D loss: 1.005713, acc.: 1.17%]
tf.Tensor(4.2613303e-13, shape=(), dtype=float32)
hi
7 [D loss: 0.961613, acc.: 1.17%]
tf.Tensor(7.102851e-09, shape=(), dtype=float32)
hi
8 [D loss: 0.732352, acc.: 50.00%]
tf.Tensor(2.3370044e-19, shape=(), dtype=float32)
hi
9 [D loss: 0.701539, acc.: 50.00%]
tf.Tensor(3.906788e-18, shape=(), dtype=float32)
hi
10 [D loss: 0.700628, acc.: 50.00%]
tf.Tensor(4.556513e-13, shape=(), dtype=float32)
hi
11 [D loss: 0.699801, acc.: 50.00%]
tf.Tensor(0.34173954, shape=(), dtype=float32)
hi
12 [D loss: 0.795090, acc.: 50.00%]
t

KeyboardInterrupt: ignored

In [ ]:
z = np.random.rand(1, 7)
img = read_image('/content/drive/MyDrive/images/9475.jpg')
img = np.expand_dims(img,axis = 0)
print(z.shape)
a = cgan.predict([img,z])
print(a)
print(da['9475'])

(1, 7)
[[0.00118032 0.00136031 0.         0.         0.00567467 0.
  0.        ]]
[1, 1, 1, 3, 2, 2, 4]
